# GDAL 的使用实例

GDAL 是使用C++编写的，然后有针对python的封装，所以其性能相对一些更易用的python库来说更好一些，所以如果进行大范围大量空间数据的运算，还是使用GDAL更好一些。

下面就日常积累一些常用到的空间数据处理代码

1. 矢量图转栅格
2. 栅格转矢量

## 矢量图转栅格

主要参考:https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#convert-an-ogr-file-to-a-raster

函数前面三个参数 Dataset dataset, int bands, Layer layer

In [1]:
from osgeo import gdal, ogr

# Define pixel_size and NoData value of new raster
pixel_size = 256
NoData_value = 255

# Filename of input OGR file
vector_fn = 'data/some_from_mxwdshld.shp'

# Filename of the raster Tiff that will be created
raster_fn = 'data/some_from_mxwdshld.tif'

# Open the data source and read in the extent
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()
x_min, x_max, y_min, y_max = source_layer.GetExtent()

# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
# 不同的格式最后出来的结果是完全不同的,所以要特别注意
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
# target_ds = gdal.GetDriverByName('HFA').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
# target_ds = gdal.GetDriverByName('MEM').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
# gdal.RasterizeLayer(target_ds, [1], source_layer)
gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[1])

0

In [2]:
ds = gdal.Open(raster_fn, 0)
band = ds.GetRasterBand(1)
cols = ds.RasterXSize
rows = ds.RasterYSize
data = band.ReadAsArray(0, 0, cols, rows)
data

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [3]:
import numpy as np
np.all(data==255)

True

如上所示,所有数据都是 SetNoDataValue 的值

In [4]:
memory_driver = gdal.GetDriverByName('MEM')
zone_mask_ds = memory_driver.Create('', x_res, y_res, 1, gdal.GDT_Byte)
zone_mask_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
zone_mask_band = zone_mask_ds.GetRasterBand(1)
zone_mask_band.SetNoDataValue(NoData_value)
gdal.RasterizeLayer(zone_mask_ds, [1], source_layer, burn_values=[1])
zone_mask = zone_mask_band.ReadAsArray(0, 0, cols, rows)
np.all(zone_mask==255)

False

In [5]:
# 可以试试,就知道这里的1就是burn values传进来的值
np.where(zone_mask==1)

(array([  0,   0,   1, ..., 654, 654, 654], dtype=int64),
 array([301, 302, 299, ..., 597, 604, 606], dtype=int64))

更多参数信息可以参考:https://www.cnblogs.com/xiaowangba/archive/2012/01/29/6314034.html

换成MEM驱动之后,可以看到就有burn_valuies的值了.

所以这里要特别注意不同的格式对应的情况,很可能最后呈现的结果是完全不同的(可以试试HFA等,结果又完全不一样),如果是自己在计算过程中要根据矢量图生成一个栅格图的mask,还是尽量使用MEM,这样会是自己想要的.

## 栅格的一个band转矢量

主要参考:https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#polygonize-a-raster-band

 gdal_polygonize 命令行工具 包装了 GDALFPolygonize 来做这件事,在python里, 使用 gdal.Polygonize 函数来完成同样的操作.

In [ ]:
from osgeo import gdal, ogr
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

#
#  get raster datasource
#
src_ds = gdal.Open( "ospy_data4/aster.img" )
if src_ds is None:
    print ('Unable to open %s' % src_filename)
    sys.exit()

try:
    srcband = src_ds.GetRasterBand(1)
except RuntimeError as e:
    # for example, try GetRasterBand(10)
    print ('Band ( %i ) not found' % band_num)
    print(e)
    sys.exit()

#
#  create output datasource
#
dst_layername = "ospy_data4/aster"
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource( dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername, srs = None )

gdal.Polygonize( srcband, None, dst_layer, 0)

Polygonize参数分别是 Band srcBand, Band maskBand,Layer outLayer, int iPixValField

根据GDAL 官方文档:https://gdal.org/api/gdal_alg.html?highlight=polygonize#_CPPv415GDALFPolygonize15GDALRasterBandH15GDALRasterBandH9OGRLayerHiPPc16GDALProgressFuncPv

- hSrcBand: the source raster band to be processed.
- hMaskBand: an optional mask band. All pixels in the mask band with a value other than zero will be considered suitable for collection as polygons.
- hOutLayer: the vector feature layer to which the polygons should be written.
- iPixValField: the attribute field index indicating the feature attribute into which the pixel value of the polygon should be written.